In [2]:
import random
from math import sqrt
from functools import lru_cache
from collections import deque

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.ent = None

    def empty(self):
        return self.ent is None

class Entity:
    def __init__(self, color):
        self.color = color

class Eater(Entity): pass

class Food(Entity):
    def __init__(self, color, is_last=False):
        super().__init__(color)
        self.is_last = is_last

class Grid:
    DIRS = [(0,1), (0,-1), (1,0), (-1,0)]
    COLOR_MAP = {
        'Food': {'White': 0, 'Green': 1, 'Blue': 2, 'Red': 3, 'Yellow': 4, 'Purple': 5, 'Pink': 6, 'Brown': 7},
        'Eater': {'Green': 0, 'Blue': 1, 'Red': 2, 'Yellow': 3, 'Purple': 4, 'Pink': 5, 'Brown': 6}
    }

    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = [[Cell(x, y) for y in range(cols)] for x in range(rows)]
        self.moves = []
        self.min_separation = 2
        self._distance_cache = {}

    def calculate_clustering_score(self):
        colors = set(self.COLOR_MAP['Food'].keys()) - {'White'}
        metrics = {}
        
        for color in colors:
            foods = [(x, y) for x in range(self.rows) for y in range(self.cols)
                    if isinstance(self.grid[x][y].ent, Food) and self.grid[x][y].ent.color == color]
            
            if not foods:
                continue

            eater_pos = next(((x, y) for x in range(self.rows) for y in range(self.cols)
                            if isinstance(self.grid[x][y].ent, Eater) and self.grid[x][y].ent.color == color), None)
            
            if not eater_pos:
                continue

            color_metrics = {
                'food_count': len(foods),
                'avg_food_separation': self._calculate_avg_separation(foods),
                'largest_cluster_size': self._find_largest_cluster(foods),
                'clusters_count': self._count_clusters(foods),
                'avg_distance_to_eater': self._calculate_avg_distance_to_point(foods, eater_pos),
                'distribution_score': self._calculate_distribution_score(foods)
            }
            
            if (color_metrics['largest_cluster_size'] == color_metrics['food_count'] and 
                color_metrics['food_count'] > 1):
                color_metrics['disqualified'] = True
            else:
                color_metrics['disqualified'] = False
                
            metrics[color] = color_metrics

        if not metrics:
            return {'final_score': float('inf'), 'color_metrics': {}, 'disqualified': True}

        disqualified = any(m['disqualified'] for m in metrics.values())
        final_score = float('inf') if disqualified else self._calculate_final_score(metrics)
        
        return {
            'final_score': final_score,
            'color_metrics': metrics,
            'disqualified': disqualified
        }

    @lru_cache(maxsize=1024)
    def _manhattan_distance(self, pos1, pos2):
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    def _calculate_avg_separation(self, positions):
        if len(positions) < 2:
            return float('inf')
        
        positions = tuple(sorted(positions))
        cache_key = positions
        if cache_key in self._distance_cache:
            return self._distance_cache[cache_key]
            
        distances = []
        for i, pos1 in enumerate(positions[:-1]):
            for pos2 in positions[i+1:]:
                distances.append(self._manhattan_distance(pos1, pos2))
        
        result = sum(distances) / len(distances) if distances else float('inf')
        self._distance_cache[cache_key] = result
        return result

    def _count_clusters(self, positions, cluster_threshold=2):
        if not positions:
            return 0
            
        positions_set = set(positions)
        clusters = 0
        
        while positions_set:
            clusters += 1
            start = next(iter(positions_set))
            positions_set.remove(start)
            
            queue = deque([start])
            while queue:
                current = queue.popleft()
                for pos in list(positions_set):
                    if self._manhattan_distance(current, pos) <= cluster_threshold:
                        queue.append(pos)
                        positions_set.remove(pos)
        
        return clusters

    def _find_largest_cluster(self, positions, cluster_threshold=2):
        if not positions:
            return 0
            
        positions_set = set(positions)
        largest_cluster = 0
        
        while positions_set:
            start = next(iter(positions_set))
            positions_set.remove(start)
            
            queue = deque([start])
            cluster_size = 1
            
            while queue:
                current = queue.popleft()
                for pos in list(positions_set):
                    if self._manhattan_distance(current, pos) <= cluster_threshold:
                        queue.append(pos)
                        positions_set.remove(pos)
                        cluster_size += 1
            
            largest_cluster = max(largest_cluster, cluster_size)
        
        return largest_cluster

    def _calculate_avg_distance_to_point(self, positions, target):
        if not positions:
            return 0
        distances = [self._manhattan_distance(pos, target) for pos in positions]
        return sum(distances) / len(distances)

    def _calculate_distribution_score(self, positions):
        if not positions:
            return 0
            
        mid_row = self.rows // 2
        mid_col = self.cols // 2
        quadrants = [0] * 4
        
        for x, y in positions:
            quad_idx = (x >= mid_row) * 2 + (y >= mid_col)
            quadrants[quad_idx] += 1
            
        avg = sum(quadrants) / 4
        variance = sum((q - avg) ** 2 for q in quadrants) / 4
        return sqrt(variance)

    def _calculate_final_score(self, metrics):
        scores = []
        for color_metrics in metrics.values():
            if color_metrics.get('disqualified', False):
                continue
                
            color_score = (
                (5 / (color_metrics['avg_food_separation'] + 0.1)) +
                (color_metrics['largest_cluster_size'] / 3) +
                (1 / (color_metrics['clusters_count'] + 0.1)) +
                (color_metrics['distribution_score']) +
                (1 / (color_metrics['avg_distance_to_eater'] + 1))
            )
            scores.append(color_score)
        
        return sum(scores) / len(scores) if scores else float('inf')
    
    def place(self, ent, x, y):
        self.grid[x][y].ent = ent

    def clear(self):
        for row in self.grid:
            for cell in row: cell.ent = None

    def gen_puzzle(self, eaters, white_pct, min_food):
        self.clear()
        self._distance_cache = {}
        eater_pos = []
        
        colors = [c for c, n in eaters.items() for _ in range(n)]
        random.shuffle(colors)
        
        for color in colors:
            pos = self.get_empty()
            if not pos: break
            x, y = pos
            e = Eater(color)
            self.place(e, x, y)
            eater_pos.append({'e': e, 'x': x, 'y': y, 'count': 0})

        active_eaters = eater_pos.copy()
        while active_eaters:
            moved_any = False
            for einfo in active_eaters[:]:
                if einfo['count'] >= min_food:
                    active_eaters.remove(einfo)
                    continue
                    
                if self.move_eater(einfo):
                    moved_any = True
                else:
                    active_eaters.remove(einfo)
            
            if not moved_any:
                break

        remaining_spaces = self.count_empty()
        active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
        random.shuffle(active_eaters)
        
        failures = 0
        max_failures = 10
        
        while remaining_spaces > 0 and active_eaters and failures < max_failures:
            moved_any = False
            
            for einfo in active_eaters[:]:
                if self.move_eater(einfo):
                    moved_any = True
                    remaining_spaces -= 1
                    failures = 0
                else:
                    active_eaters.remove(einfo)
                    
                if remaining_spaces <= 0:
                    break
            
            if not moved_any:
                failures += 1
            
            active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
            random.shuffle(active_eaters)

        self.whiten_food(white_pct)

    def can_move_eater(self, einfo):
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            if new_x is not None and (new_x != x or new_y != y):
                return True
        return False

    def move_eater(self, einfo):
        random.shuffle(self.DIRS)
        best_move = None
        best_score = float('-inf')
        
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            
            if new_x is None or (new_x == x and new_y == y):
                continue
            
            old_ent = self.grid[x][y].ent
            self.grid[x][y].ent = Food(einfo['e'].color)
            
            move_score = self._evaluate_food_placement(x, y, einfo['e'].color)
            
            self.grid[x][y].ent = old_ent
            
            if move_score > best_score:
                best_score = move_score
                best_move = (new_x, new_y)
        
        if best_move is None:
            for dx, dy in self.DIRS:
                x, y = einfo['x'], einfo['y']
                new_x, new_y = self.get_max_pos(x, y, dx, dy)
                if new_x is not None and (new_x != x or new_y != y):
                    best_move = (new_x, new_y)
                    break
        
        if best_move:
            new_x, new_y = best_move
            x, y = einfo['x'], einfo['y']
            
            is_first_food = einfo['count'] == 0
            
            self.grid[x][y].ent = None
            self.place(einfo['e'], new_x, new_y)
            self.place(Food(einfo['e'].color, is_last=is_first_food), x, y)
            einfo['count'] += abs(new_x - x) + abs(new_y - y)
            
            self.moves.append({
                'eater_color': einfo['e'].color,
                'start': (new_x, new_y),
                'end': (x, y),
                'food_color': einfo['e'].color,
                'dist': abs(new_x - x) + abs(new_y - y)
            })
            
            einfo['x'], einfo['y'] = new_x, new_y
            return True
            
        return False
    
    def _evaluate_food_placement(self, x, y, color):
        same_color_foods = [(fx, fy) for fx in range(self.rows) for fy in range(self.cols)
                           if isinstance(self.grid[fx][fy].ent, Food) 
                           and self.grid[fx][fy].ent.color == color]
        
        if not same_color_foods:
            return 100
            
        min_distance = min(self._manhattan_distance((x, y), (fx, fy)) for fx, fy in same_color_foods)
        
        temp_foods = same_color_foods + [(x, y)]
        
        cluster_size = self._find_largest_cluster(temp_foods)
        clusters_count = self._count_clusters(temp_foods)
        
        if cluster_size == len(temp_foods) and len(temp_foods) > 1:
            return -1000
            
        separation_score = 0
        if min_distance >= self.min_separation:
            separation_score = 50 + min_distance * 10
        else:
            separation_score = 20 - (self.min_separation - min_distance) * 15
            
        cluster_score = clusters_count * 20
        
        return separation_score + cluster_score

    def get_max_pos(self, x, y, dx, dy):
        nx, ny = x, y
        while self.in_bounds(nx + dx, ny + dy) and self.grid[nx + dx][ny + dy].empty():
            nx += dx
            ny += dy
        return (nx, ny) if (nx, ny) != (x, y) else (None, None)

    def whiten_food(self, pct):
        food = [(c.x, c.y) for row in self.grid for c in row 
                if isinstance(c.ent, Food) and c.ent.color != 'White' and not c.ent.is_last]
        to_white = random.sample(food, int(len(food) * pct / 100)) if food else []
        for x, y in to_white:
            self.grid[x][y].ent.color = 'White'

    def get_empty(self):
        empty = [(x,y) for x in range(self.rows) for y in range(self.cols) 
                if self.grid[x][y].empty()]
        return random.choice(empty) if empty else None

    def in_bounds(self, x, y):
        return 0 <= x < self.rows and 0 <= y < self.cols

    def count_empty(self):
        return sum(1 for row in self.grid for cell in row if cell.empty())

    def to_scene(self, level_id):
        width, height = 1440, 2560
        step = max(180, width // self.cols - 24)
        
        start_x = width/2 - step*self.cols/2 + step*0.5
        start_y = height/2 - step*self.rows/2 + step*0.5 - 40

        scene = f"""[gd_scene load_steps=4 format=3 uid="{level_id}"]

[ext_resource type="Script" path="res://Levels/Level.cs" id="lvlid"]
[ext_resource type="PackedScene" uid="uid://byatslmwbvorg" path="res://Entities/Food/Food.tscn" id="foodid"]
[ext_resource type="PackedScene" uid="uid://dd570jgysudow" path="res://Entities/Eater/Eater.tscn" id="eaterid"]

[node name="Level" type="Node"]
script = ExtResource("lvlid")
{self.solution_metadata()}

[node name="Food" type="Node" parent="."]

[node name="Eaters" type="Node" parent="."]
"""
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                pos_x = start_x + y*step
                pos_y = start_y + x*step
                
                if isinstance(cell.ent, Food):
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = {self.COLOR_MAP['Food'][cell.ent.color]}
IsLast = {"true" if cell.ent.is_last else "false"}
"""
                elif isinstance(cell.ent, Eater):
                    scene += f"""
[node name="Eater{x}_{y}" parent="Eaters" instance=ExtResource("eaterid")]
position = Vector2({pos_x}, {pos_y})
EaterType = {self.COLOR_MAP['Eater'][cell.ent.color]}
BoardStatePositionId = Vector2i({x}, {y})
ValidFoodTypes = [0, {self.COLOR_MAP['Food'][cell.ent.color]}]
"""
                elif cell.ent is None:
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = 0
IsLast = false
"""
        return scene

    def solution_metadata(self):
        metadata = "metadata/solution = PackedVector2Array("
        for move in self.moves:
                metadata += f"{move["start"][0]}, {move["start"][1]}, {move["end"][0]}, {move["end"][1]}, "
        metadata = metadata[:-2] + ")"
        return metadata

def gen_boards(num_select, total_gen, rows, cols, eaters, white_pct, min_food):
    boards = []
    attempts = 0
    max_tries = total_gen * 100

    while len(boards) < total_gen and attempts < max_tries:
        g = Grid(rows, cols)
        g.gen_puzzle(eaters, white_pct, min_food)
        
        empty_count = g.count_empty()
        
        if empty_count == 0 and all(
            sum(1 for row in g.grid for cell in row 
                if isinstance(cell.ent, Food) and cell.ent.color == color) >= min_food 
            for color in eaters):
            
            clustering_info = g.calculate_clustering_score()
            
            # Skip disqualified boards
            if clustering_info['disqualified']:
                attempts += 1
                continue
                
            boards.append({
                'board': g,
                'moves': g.moves.copy(),
                'empty': empty_count,
                'clustering_score': clustering_info['final_score'],
                'metrics': clustering_info['color_metrics']
            })
        attempts += 1

    boards.sort(key=lambda x: x['clustering_score'])
    return boards[:num_select]


# Example usage
if __name__ == "__main__":
    # Configuration
    DIFFICULTY = {
        'tutorial': ({'Green': 1}, 3, 3, 2, 30, 1),
        'very_easy': ({'Blue': 1, 'Green': 1}, 4, 4, 3, 30, 2),
        'very_easy2': ({'Blue': 1, 'Green': 1}, 5, 4, 10, 30, 2),
        'easy': ({'Blue': 1, 'Green': 1, 'Red': 1}, 5, 5, 15, 30, 3),
        'easy2': ({'Blue': 1, 'Green': 1, 'Red': 1}, 6, 5, 15, 30, 3),
        'medium': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 6, 6, 20, 20, 3),
        'medium2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 6, 20, 20, 3),
        'hard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 7, 7, 20, 20, 3),
        'hard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 20, 20, 4),
        'veryhard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 20, 20, 4),
        'veryhard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 20, 20, 4),
        'max': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 20, 15, 4),
        'max2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 20, 15, 5)
    }

    lvlid = 1
    for diff, (eaters, rows, cols, num_select, white_pct, min_food) in DIFFICULTY.items():
        new_boards = gen_boards(num_select, 5000, rows, cols, eaters, white_pct, min_food)
        print(f"Generated {len(new_boards)} new boards for difficulty {diff} with average score of {sum([board["clustering_score"] for board in new_boards])/len(new_boards)}")
        for board in new_boards:
            with open(f"../Levels/Level{lvlid}.tscn", "w") as f:
                f.write(board["board"].to_scene(lvlid))
            lvlid += 1

    # Save boards


KeyboardInterrupt

